In [18]:
import numpy as np
import pandas as pd
from sklearn import pipeline
from sklearn import preprocessing
from ydata_profiling import ProfileReport

In [19]:
dtypes = {
'user':                     str,
'gender':                   str,
'age':                 np.int64,
'how_tall_in_meters':np.float64,
'weight':              np.int64,
'body_mass_index':   np.float64,
'x1':                  np.int64,
'y1':                  np.int64,
'z1':                  np.int64,
'x2':                  np.int64,
'y2':                  np.int64,
'z2':                  np.int64,
'x3':                  np.int64,
'y3':                  np.int64,
'z3':                  np.int64,
'x4':                  np.int64,
'y4':                  np.int64,
'z4':                  np.int64,
'class':                    str,
}
def errorproofZ4(x):
    try:
        return np.int64(x)
    except ValueError:
        return np.NaN
df = pd.read_csv("dataset-HAR-PUC-Rio.csv",sep=';',dtype=dtypes,
converters={
    'how_tall_in_meters':lambda x: np.float64(x.replace(',','.')),
    'body_mass_index':lambda x: np.float64(x.replace(',','.')),
    'z4': errorproofZ4})

profile = ProfileReport(df[['x1','y1','z1','x2','y2','z2','x3','y3','z3','x4','y4','z4',]], title="HAR-PUC measurements profile")

C:\Users\tomi\AppData\Local\Temp\ipykernel_27688\1030740947.py:27: ParserWarning: Both a converter and dtype were specified for column how_tall_in_meters - only the converter will be used.
  df = pd.read_csv("dataset-HAR-PUC-Rio.csv",sep=';',dtype=dtypes,
C:\Users\tomi\AppData\Local\Temp\ipykernel_27688\1030740947.py:27: ParserWarning: Both a converter and dtype were specified for column body_mass_index - only the converter will be used.
  df = pd.read_csv("dataset-HAR-PUC-Rio.csv",sep=';',dtype=dtypes,
C:\Users\tomi\AppData\Local\Temp\ipykernel_27688\1030740947.py:27: ParserWarning: Both a converter and dtype were specified for column z4 - only the converter will be used.
  df = pd.read_csv("dataset-HAR-PUC-Rio.csv",sep=';',dtype=dtypes,


In [20]:
profile.to_widgets()

d:\U_targyak\ERASMUS\AI\hf\.venv\lib\site-packages\ydata_profiling\utils\dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
#drop the only bad value
df.dropna(inplace=True)

In [ ]:
full_profile =  ProfileReport(df, title="Full HAR-PUC profile")
full_profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

## Scaling
It is clearly visible, that the data is from a few people.
The scale of the variables is very different, but most of them are not very skewed, they mainly show a gaussian distribution, or gaussina mixture distributions.
Some of the data seems to be truncated.
## Standard Scaling 
To have all of the data in the same range the measured variables could be transformed to a standard value.
Also the user values should be transformed to Standard range, as they are related to the scale of the measured variables. But due to the low number of users only one variable to represent them would be enough, so variables like gender or age or name could be easily dropped.

## calss conversion
Calsses should be converted to integer as well

In [22]:
df.columns

Index(['user', 'gender', 'age', 'how_tall_in_meters', 'weight',
       'body_mass_index', 'x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3',
       'x4', 'y4', 'z4', 'class'],
      dtype='object')

In [23]:
ids = {j:i for i,j in enumerate(df['class'].unique())}
ids

{'sitting': 0, 'sittingdown': 1, 'standing': 2, 'standingup': 3, 'walking': 4}

In [26]:
cleaned = df.copy()
cleaned["class"] = df["class"].apply(lambda x: ids[x])
cleaned.drop(["age","user","gender"],axis=1)

,how_tall_in_meters,weight,body_mass_index,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,y4,z4,class
0,1.62,75,28.6,-3,92,-63,-23,18,-19,5,104,-92,-150,-103,-147.0,0
1,1.62,75,28.6,-3,94,-64,-21,18,-18,-14,104,-90,-149,-104,-145.0,0
2,1.62,75,28.6,-1,97,-61,-12,20,-15,-13,104,-90,-151,-104,-144.0,0
3,1.62,75,28.6,-2,96,-57,-15,21,-16,-13,104,-89,-153,-103,-142.0,0
4,1.62,75,28.6,-1,96,-61,-13,20,-15,-13,104,-89,-153,-104,-143.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165628,1.67,67,24.0,2,93,-148,-8,88,-100,15,120,-100,-189,-77,-160.0,4
165629,1.67,67,24.0,-1,94,-147,-17,87,-99,17,121,-99,-190,-78,-158.0,4
165630,1.67,67,24.0,-10,93,-143,-19,86,-104,0,114,-101,-185,-80,-153.0,4
165631,1.67,67,24.0,-15,86,-152,-19,88,-117,-44,155,-25,-185,-84,-156.0,4


In [27]:
cleaned.to_parquet("unscaled_train_data.parquet")